# cuProj Python User's Guide

cuProj is a GPU-accelerated Python library for cartographic coordinate projection and coordinate transformations between coordinate reference systems (CRS). The cuProj Python API provides an accessible interface to high-performance projections accelerated by CUDA-enabled GPUs. The API closely follows the [PyProj](https://pyproj4.github.io/pyproj/stable/) API.

## Contents

This guide provides a working example for all of the python API components of cuProj.  
The following list links to each subsection.

* [Installing cuProj](#Installing-cuProj)
* [Transformations with Transformer](#Transformations-with-Transformer)

## Installing cuProj
Read the [RAPIDS Quickstart Guide](  https://rapids.ai/start.html     ) to learn more about installing all RAPIDS libraries, including cuProj.

If you are working on a system with a CUDA-enabled GPU and have CUDA installed, uncomment the  
following cell and install cuSpatial:

In [1]:
# !conda create -n rapids-23.08 -c rapidsai -c conda-forge -c nvidia \ 
#     cuproj=23.08 python=3.10 cudatoolkit=12.0

For other options to create a RAPIDS environment, such as docker or build from source, see  
[RAPIDS Release Selector](  https://rapids.ai/start.html#get-rapids). 

If you wish to contribute to cuProj, you should create a source build using the included
devcontainer. Simply clone the github repository and open the folder in VSCode. VSCode will prompt
you to open the folder in a devcontainer.

## Transformations with Transformer

The primary class in cuProj is the `Transformer` class, which is used to transform coordinates from one CRS to another. The `Transformer` class is created from a source CRS and a destination CRS, which can be specified using a CRS string, an EPSG code, or an `(<authority>, code)` tuple. The `Transformer` class can then be used to transform coordinates from the source CRS to the destination CRS.

Currently only the EPSG authority is supported, and only a subset of the EPSG codes are supported. The following EPSG codes are supported:

- WGS84 (EPSG:4326)
- UTM (EPSG:32600-32660 and EPSG:32700-32760)

The following simple example transforms a single (lat, lon) coordinate from WGS84 to UTM.

In [2]:
from cuproj.transformer import Transformer

# Sydney opera house latitude and longitude
lat = -33.8587
lon = 151.2140

# Transform to UTM (x, y) in meters using CuProj
cu_transformer = Transformer.from_crs("epsg:4326", "EPSG:32756")
x, y = cu_transformer.transform(lat, lon)

print(f"WGS84 (lat,lon): ({lat:.2f}, {lon:.2f}) degrees")
print(f"UTM Zone 56S (x,y): ({x:.2f}, {y:.2f}) meters")


WGS84 (lat,lon): (-33.86, 151.21) degrees
UTM Zone 56S (x,y): (334783.95, 6252075.96) meters


But cuProj really shines when you have a large number of points to transform. The following code transforms 10,000 (lat, lon) points in a grid around London, UK.

In [4]:
import cupy as cp

# Box around Sydney, NSW, Australia
min_corner = [150.502, -34.118]
max_corner = [151.343, -33.578]
crs_to = "EPSG:32756"

num_points_x = 100
num_points_y = 100

# A grid of 100x100 points in the bounding box of London in WGS84 (lat/lon)
# stored as a list of two arrays (x, y) in device memory (cupy)
x, y = cp.meshgrid(
    cp.linspace(min_corner[0], max_corner[0], num_points_y),
    cp.linspace(min_corner[1], max_corner[1], num_points_x))
grid = [x.reshape(-1), y.reshape(-1)]

transformer = Transformer.from_crs("EPSG:4326", crs_to)
x, y = transformer.transform(*grid)

print(f"min_corner in UTM zone 30N: ({x[0]}, {y[0]}) in meters")
print(f"max_corner in UTM zone 30N: ({x[-1]}, {y[-1]}) in meters")

min_corner in UTM zone 30N: (674278.2944749976, 5686155.048018296) in meters
max_corner in UTM zone 30N: (728531.6016852399, 5721710.805206091) in meters


In [5]:
x[-1] - x[1]

array(54264.78809779)

In [6]:
y[-1] - y[0]

array(35555.75718779)